<a href="https://colab.research.google.com/github/Castcocoon/Python_practice/blob/main/Kaggle_House_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KaggleのHouse Pricesに取り組む 2021-07-12
kaggle URL:https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv

In [ ]:
import numpy as np
import pandas as pd
import io
from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
# データの読み込み
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

In [ ]:
df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,Comp

In [ ]:
#データ結合
df_all = df_train.append(df_test)
# 後で分けるときのためにフラッグをつけておく
df_train["TrainFlag"] = True
df_test["TrainFlag"] = False

df_all.index = df_all['Id']
df_all.drop('Id', axis = 1, inplace=True)

In [ ]:
# ダミー変数化
df_all = pd.get_dummies(df_all, drop_first= True)

k個のカテゴリーをダミー化する場合、k-1個のダミー変数があればよいが、get_dummies()関数ではデフォルトでk個のダミー変数に変換される。

引数drop_first=Trueとすると、最初のカテゴリーが除外されk-1個のダミー変数に変換される。

# データ分割

In [ ]:
df_train = df_all[df_all['TrainFlag'] == True]
df_train = df_train.drop(['TrainFlag'], axis = 1)

df_test = df_all[df_all['TrainFlag'] == False]
df_test = df_test.drop(['TrainFlag'], axis = 1)
df_test = df_test.drop(['SalePrice'], axis=1)

In [ ]:
y = df_train['SalePrice'].values
x = df_train.drop(['SalePrice'], axis = 1).values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 100)

# XGBoostを使う

In [ ]:
dtrain = xgb.DMatrix(x_train, label = y_train)
dvalid = xgb.DMatrix(x_test, label = y_test)
dtest = xgb.DMatrix(df_test.values)

In [ ]:
params = {
    'objective': 'reg:squarederror', 'silent':1, 'random_state':100, 'eval_metric': 'rmse'
}
num_round = 500
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

In [ ]:
model = xgb.train(params, 
                  dtrain,
                  num_round,
                  early_stopping_rounds = 20,
                  evals = watchlist)

[0]	train-rmse:141273	eval-rmse:144148
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[1]	train-rmse:101982	eval-rmse:106133
[2]	train-rmse:74388.2	eval-rmse:80308
[3]	train-rmse:54707.7	eval-rmse:62344.8
[4]	train-rmse:41004.6	eval-rmse:50993.1
[5]	train-rmse:30976.8	eval-rmse:43892
[6]	train-rmse:24056.6	eval-rmse:39549.8
[7]	train-rmse:19153	eval-rmse:36594.5
[8]	train-rmse:15828.7	eval-rmse:34675.5
[9]	train-rmse:13520.7	eval-rmse:33859.2
[10]	train-rmse:11915.1	eval-rmse:33370.8
[11]	train-rmse:10787.6	eval-rmse:33003
[12]	train-rmse:10015.1	eval-rmse:32552.8
[13]	train-rmse:9427.55	eval-rmse:32346.7
[14]	train-rmse:8914.3	eval-rmse:32231.3
[15]	train-rmse:8492.26	eval-rmse:32043.6
[16]	train-rmse:8219.59	eval-rmse:31912.6
[17]	train-rmse:7944.01	eval-rmse:31757.8
[18]	train-rmse:7701.38	eval-rmse:31721.3
[19]	train-rmse:7487.12	eval-rmse:31641.1
[20]	train-rmse:7349.09	eval-rmse:31621


In [ ]:
prediction = model.predict(dtest, ntree_limit = model.best_ntree_limit)

prediction = np.round(prediction)

In [ ]:
submission = pd.DataFrame({"id": df_test.index, "SalePrice": prediction})
